Given e1_pos, e2_pos, sentence. Update the sentence to say DRUG and OTHER_DRUG and update the positions

'The bear_e1 ran home_e2 to_e2 the mountains'

In [28]:
%load_ext autoreload

In [36]:
%autoreload
import sys
sys.path.append('..')
import relation_extraction.data.utils as utils
import os

RESOURCE_PATH = "/data/medg/misc/semeval_2010/medical-data/DDICorpus/pre-processed/extraction/"
def res(path): return os.path.join(RESOURCE_PATH, path)

[nltk_data] Downloading package wordnet to
[nltk_data]     /afs/csail.mit.edu/u/g/geeticka/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [1]:
sentence = 'The bear ran home to the mountains'

In [2]:
e1_pos = (1,1)
e2_pos = (3,4)

In [18]:
# given a tokenized and splitted sentence
def sentence_replace(sentence, positions, string_update):
    return sentence[:positions[0]] + [string_update] + sentence[positions[1]+1:]

In [20]:
sentence_replace(['The', 'bear', 'ran', 'home', 'to', 'the', 'mountains'], (3,4), 'DRUG')

['The', 'bear', 'ran', 'DRUG', 'the', 'mountains']

In [24]:
# sentence is the sentence to update and entity positions is a list of entity positions
def per_sentence_replacement_ddi(sentence, entity_positions):
    # if entity position is updated, then all positions after it also have to be updated
    sentence = sentence.split() # no need to do this for the code
    
    e0_pos = entity_positions[0]
    sentence = sentence_replace(sentence, e0_pos, 'DRUG1')
    new_e0_pos = (e0_pos[0], e0_pos[0])
   
    entity_positions[0] = new_e0_pos
    diff = e0_pos[1] - e0_pos[0] # if the entity is 2 word, then move every other e_pos down by 1
    if diff > 0:
        for i in range(1, len(entity_positions)):
            e_pos = entity_positions[i]
            if e_pos[0] > e0_pos[1]:
                entity_positions[i] = (entity_positions[i][0] - diff, entity_positions[i][1] - diff)
     
    e1_pos = entity_positions[1]
    sentence = sentence_replace(sentence, e1_pos, 'DRUG2')
    new_e1_pos = (e1_pos[0], e1_pos[0])
    
    entity_positions[1] = new_e1_pos
    diff = e1_pos[1] - e1_pos[0]
    if diff > 0 and len(entity_positions) > 2:
        for i in range(2, len(entity_positions)):
            e_pos = entity_positions[i]
            if e_pos[0] > e1_pos[1]:
                entity_positions[i] = (entity_positions[i][0] - diff, entity_positions[i][1] - diff)
    # then should handle for the case when there are more than entity 1 and entity 2 i.e. drug0 (any other drug)
    return sentence, entity_positions

In [27]:
per_entity_replacement('The bear ran home to the mountains', [(1,2), (3,4)])

(['The', 'DRUG0', 'DRUG1', 'the', 'mountains'], [(1, 1), (2, 2)])

In [41]:
train_data = open(res('train.txt'))

train_data = utils.split_data_cut_sentence(train_data)

In [42]:
new_train_data = utils.replace_by_drug_ddi(train_data)

In [48]:
indexes = []
sentences, relations, e1_pos, e2_pos = new_train_data
for i in range(len(sentences)):
    sent = sentences[i]
    pos1 = e1_pos[i]
    pos2 = e2_pos[i]
    sentence_len = len(sent)
    if(pos1[0] >= sentence_len or pos2[0] >= sentence_len):
        print(len(sent), i)
        indexes.append(i)
        print(sent, pos1, pos2)

22 8106
['in', 'some', 'patients', ',', 'the', 'administration', 'of', 'indocin', 'can', 'reduce', 'the', 'diuretic', ',', 'natriuretic', ',', 'and', 'antihypertensive', 'effects', 'of', 'DRUG1', '.', 'DRUG2'] (19, 19) (26, 26)
24 8107
['in', 'some', 'patients', ',', 'the', 'administration', 'of', 'indocin', 'can', 'reduce', 'the', 'diuretic', ',', 'natriuretic', ',', 'and', 'antihypertensive', 'effects', 'of', 'loop', ',', 'DRUG1', '.', 'DRUG2'] (21, 21) (26, 26)
37 9302
['non', '-', 'steroidal', 'anti', '-', 'inflammatory', 'drugs', ':', 'in', 'some', 'patients', ',', 'the', 'administration', 'of', 'a', 'non', '-', 'steroidal', 'anti', '-', 'inflammatory', 'agent', 'can', 'reduce', 'the', 'diuretic', ',', 'natriuretic', ',', 'and', 'antihypertensive', 'effects', 'of', 'DRUG1', '.', 'DRUG2'] (34, 34) (40, 40)
39 9303
['non', '-', 'steroidal', 'anti', '-', 'inflammatory', 'drugs', ':', 'in', 'some', 'patients', ',', 'the', 'administration', 'of', 'a', 'non', '-', 'steroidal', 'anti', '

In [49]:
sentences, relations, e1_pos, e2_pos = train_data
for i in range(len(sentences)):
    if i in indexes:
        print(sentences[i], e1_pos[i], e2_pos[i])

['in', 'some', 'patients', ',', 'the', 'administration', 'of', 'indocin', 'can', 'reduce', 'the', 'diuretic', ',', 'natriuretic', ',', 'and', 'antihypertensive', 'effects', 'of', 'loop', ',', 'potassium', '-', 'sparing', ',', 'and', 'thiazide', 'diuretics', '.'] (19, 27) (26, 27)
['in', 'some', 'patients', ',', 'the', 'administration', 'of', 'indocin', 'can', 'reduce', 'the', 'diuretic', ',', 'natriuretic', ',', 'and', 'antihypertensive', 'effects', 'of', 'loop', ',', 'potassium', '-', 'sparing', ',', 'and', 'thiazide', 'diuretics', '.'] (21, 27) (26, 27)
['non', '-', 'steroidal', 'anti', '-', 'inflammatory', 'drugs', ':', 'in', 'some', 'patients', ',', 'the', 'administration', 'of', 'a', 'non', '-', 'steroidal', 'anti', '-', 'inflammatory', 'agent', 'can', 'reduce', 'the', 'diuretic', ',', 'natriuretic', ',', 'and', 'antihypertensive', 'effects', 'of', 'loop', ',', 'potassium', '-', 'sparing', 'and', 'thiazide', 'diuretics', '.'] (34, 41) (40, 41)
['non', '-', 'steroidal', 'anti', '-'